In [ ]:
from collections import Counter
from matplotlib import pyplot as plt 
from rich import print
import google.generativeai as genai
import json
import tabulate
import sf
import secret_constants
import textwrap

Make sure you download the right [bulk file from the scryfall api page](https://scryfall.com/docs/api/bulk-data) and reference it here. I tried grabbing it automatically, but there is a significant size mismatch from the interactively downloaded one and the files listed in `/bulk-data`.

In [ ]:

scryfall_all_cards = "all-cards-20240412091807.json"
cards = sf.elligible_cards(scryfall_all_cards)
print("Pulled in ", len(cards), " valid creature cards.")
animal_pools = sf.animal_pools(cards)

`cards` will be all the raw cards used in scope for this (commander-legal animals that exist in real life).

In [ ]:
legends = []
legend_creature_types = []
for card in cards:
    if "frame_effects" in card:
        if "legendary" in card["frame_effects"]:
            legends.append(card)
            legend_creature_types.extend(sf.allowed_type_line(card))

print("Pulled in ", len(legends), " legendary creatures with ", len(set(legend_creature_types)), " unique, elligible creature types.")
print("Example card:")
print(legends[0])

In [ ]:
allowed_keys = ["id", "name", "type_line", "keywords", "oracle_text"]
legends_decision_context = json.dumps([{key: value for key, value in dict.items() if key in allowed_keys} for dict in legends])


In [ ]:
genai.configure(api_key=secret_constants.gemini_key)
model = genai.GenerativeModel('gemini-1.5-pro-latest')

prompt = """
Here is a list of magic the gathering cards and their data. Determine which of these cards, in pairs, have the most synergy and potential to work well together in a game of magic the gathering. Score each pair by how well they synergize, on a scale of 1-100 where 100 is the highest synergy. Reference each card by its id field. Make sure to find at least 20 pair combinations among these cards. Return your results as a JSON list with the following schema:

[{"synergy":int, "card1_id":str, "card2_id":str}]

All fields are required.

Important: Only return a single piece of valid JSON text.

Here are the cards:

"""
# Gemini times out because it sucks
for i in range(0,3):
    while True:
        try:
            response = model.generate_content(textwrap.dedent(prompt) + legends_decision_context)
        except:
            continue
        break

response.text


In [ ]:
response_json = json.loads(response.text.strip('`\r\n ').removeprefix('json'))
#print(json.dumps(json.loads(response_json), indent=4))

class pairing:
    def __init__(self, pair):
        self.pair = pair
        self.synergy = pair["synergy"]
        self.card1 = next((card for card in cards if card['id'] == pair["card1_id"]), None)
        self.card1_link = self.make_clickable(self.card1['scryfall_uri'], "CLICK_HERE")
        self.card2 = next((card for card in cards if card['id'] == pair["card2_id"]), None)
        self.card2_link = self.make_clickable(self.card2['scryfall_uri'], "CLICK_HERE")
        self.animal_types = sf.allowed_type_line(self.card1).union(sf.allowed_type_line(self.card2))
        self.name = str(self.card1["name"] + " & " + self.card2["name"])
        self.colors()
        self.animal_pool()

    def colors(self):
        all_colors = list(set(self.card1["colors"]) | set(self.card2["colors"]))
        return all_colors
    
    def animal_pool(self):
        #animal_types = sf.allowed_type_line(self.card1).union(sf.allowed_type_line(self.card2))
        animals_and_their_counts = {}
        # Yo this kinda gorss tho
        for animal in self.animal_types:
            animals_and_their_counts[animal] = 0
            for self_color in self.colors():
                for all_colors_for_that_animal in animal_pools[animal]:
                    if self_color in all_colors_for_that_animal or "N" in all_colors_for_that_animal:
                        animals_and_their_counts[animal] += 1
        return animals_and_their_counts
    
    def make_clickable(self, url, name):
        return '<a href="{}">{}</a>'.format(url,name)
        
table = []
for pair in response_json:
    row = pairing(pair)
    table.append([row.synergy, row.name, row.animal_types, row.colors(), row.card1_link, row.card2_link, row.animal_pool()])

tabulate.tabulate(table, tablefmt='unsafehtml', headers=["Synergy", "Cards", "Animals", "Colors", "Card 1 Link", "Card 2 Link", "Animal Pool"])